In [ ]:
# import os
# import glob
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# import joblib
# import math

# # ==========================================
# # 1. CONFIGURATION
# # ==========================================
# # Point this to the folder containing your zonal files (e.g. east_zone_prices_wheat.csv)
# dataset_root = 'Center_Price_Dataset' 

# # Output directories
# models_root = 'models'
# graphs_root = 'graphs'

# # Training parameters
# prediction_days = 30
# future_days = 180
# epochs = 15 
# batch_size = 32

# # ==========================================
# # 2. FIND ALL CSV FILES
# # ==========================================
# csv_files = glob.glob(f"{dataset_root}/*.csv")
# print(f"Found {len(csv_files)} commodity files to process.")

# # ==========================================
# # 3. MAIN PROCESSING LOOP
# # ==========================================
# for file_path in csv_files:
#     # Extract commodity name (e.g., 'east_zone_prices_wheat' -> 'Wheat')
#     filename = os.path.basename(file_path)
#     # Simple logic to get commodity name from your specific filename format
#     # Adjust split logic if your filenames vary
#     commodity_name = filename.split('_')[-1].replace('.csv', '').capitalize() 
    
#     print(f"\n==========================================")
#     print(f" LOADING MASTER FILE: {commodity_name}")
#     print(f"==========================================")

#     try:
#         # Load the Master CSV
#         master_df = pd.read_csv(file_path)
        
#         # Standardize column names (lowercase, strip spaces)
#         master_df.columns = [c.lower().strip() for c in master_df.columns]

#         # Ensure required columns exist. Note: Your file uses 'price', not 'value'
#         if 'date' not in master_df.columns or 'price' not in master_df.columns:
#             print(f"❌ Skipping {filename}: Column names mismatch. Found: {master_df.columns}")
#             continue

#         # Parse dates
#         master_df['date'] = pd.to_datetime(master_df['date'], errors='coerce')
#         master_df = master_df.dropna(subset=['date'])

#         # Get unique centers in this file
#         centers = master_df['centre_name'].unique()
#         print(f"Found {len(centers)} centers: {centers}")

#         # ==========================================
#         # 4. CENTER-SPECIFIC TRAINING
#         # ==========================================
#         for center in centers:
#             print(f"\n--- Training for {center} ({commodity_name}) ---")
            
#             # Filter data for this specific center
#             df_center = master_df[master_df['centre_name'] == center].copy()
            
#             # Sort and set index
#             df_center = df_center.sort_values('date').set_index('date')
            
#             # Keep only price column and rename to 'value' for consistency
#             df_center = df_center[['price']].rename(columns={'price': 'value'})
            
#             # Handle missing dates (Daily Frequency)
#             df_center = df_center.asfreq('D', method='ffill')
#             df_center = df_center.dropna()

#             # Skip if not enough data points (Need at least prediction_days + input for 1 batch)
#             if len(df_center) < (prediction_days + 60):
#                 print(f"⚠️ Skipping {center}: Insufficient data ({len(df_center)} points)")
#                 continue

#             # Dynamic Train/Test Split (80% Train, 20% Test)
#             # We use dynamic split because different centers have different start dates
#             split_idx = int(len(df_center) * 0.8)
#             train_data = df_center.iloc[:split_idx]
#             test_data = df_center.iloc[split_idx:]

#             # --- Scaling ---
#             scaler = MinMaxScaler(feature_range=(0, 1))
#             scaled_train = scaler.fit_transform(train_data.values)

#             # --- Create Training Sequences ---
#             x_train, y_train = [], []
#             for x in range(prediction_days, len(scaled_train)):
#                 x_train.append(scaled_train[x-prediction_days:x, 0])
#                 y_train.append(scaled_train[x, 0])

#             x_train, y_train = np.array(x_train), np.array(y_train)
#             x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

#             # --- Build LSTM Model ---
#             model = Sequential()
#             model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
#             model.add(LSTM(units=50, return_sequences=False))
#             model.add(Dense(units=25))
#             model.add(Dense(1))
#             model.compile(optimizer='adam', loss='mean_squared_error')

#             # --- Train ---
#             model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

#             # --- Save Artifacts ---
#             # Create directory: models/Wheat/Araria/
#             save_dir = os.path.join(models_root, commodity_name, center)
#             os.makedirs(save_dir, exist_ok=True)

#             joblib.dump(scaler, os.path.join(save_dir, 'scaler.pkl'))
#             model.save(os.path.join(save_dir, 'model.keras'))

#             # --- Evaluation & Prediction ---
#             # Combine required history + test data
#             total_data = pd.concat((train_data['value'], test_data['value']), axis=0)
#             model_inputs = total_data[len(total_data) - len(test_data) - prediction_days:].values
#             model_inputs = model_inputs.reshape(-1, 1)
#             model_inputs = scaler.transform(model_inputs)

#             x_test = []
#             if len(model_inputs) > prediction_days:
#                 for x in range(prediction_days, len(model_inputs)):
#                     x_test.append(model_inputs[x-prediction_days:x, 0])

#                 x_test = np.array(x_test)
#                 x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

#                 predicted_prices = model.predict(x_test)
#                 predicted_prices = scaler.inverse_transform(predicted_prices)

#                 # --- Future Forecast ---
#                 future_predictions = []
#                 # Start with the very last available data window
#                 curr_batch = model_inputs[-prediction_days:].reshape(1, prediction_days, 1)

#                 for i in range(future_days):
#                     pred_one = model.predict(curr_batch, verbose=0)[0]
#                     future_predictions.append(pred_one)
#                     curr_batch = np.append(curr_batch[:, 1:, :], [[pred_one]], axis=1)

#                 future_predictions = scaler.inverse_transform(future_predictions)
                
#                 # Generate future dates
#                 last_date = df_center.index[-1]
#                 future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_days)

#                 # --- Plotting ---
#                 graph_save_dir = os.path.join(graphs_root, commodity_name)
#                 os.makedirs(graph_save_dir, exist_ok=True)
                
#                 plt.figure(figsize=(12, 6))
#                 # Plot last year of actual data to keep chart readable
#                 plt.plot(test_data.index, test_data['value'], color='black', label="Actual Price")
#                 plt.plot(test_data.index[:len(predicted_prices)], predicted_prices, color='green', label="Test Predictions")
#                 plt.plot(future_dates, future_predictions, color='red', linestyle='--', label="6-Month Forecast")
                
#                 plt.title(f"{commodity_name} Price Forecast - {center}")
#                 plt.xlabel("Date")
#                 plt.ylabel("Price")
#                 plt.legend()
#                 plt.grid(True)
                
#                 plt.savefig(os.path.join(graph_save_dir, f"{center}_forecast.png"))
#                 plt.close()
                
#                 print(f"✅ Saved model and graph for {center}")

#     except Exception as e:
#         print(f"❌ Critical Error processing file {file_path}: {e}")

# print("\nAll centers processed.")

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score  # <--- Added r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib
import math

# ==========================================
# 1. CONFIGURATION
# ==========================================
dataset_root = 'Center_Price_Dataset' 
models_root = 'models'
graphs_root = 'graphs'

# Training parameters
prediction_days = 30  # Optimized for smaller datasets
future_days = 180
epochs = 15 
batch_size = 32

# ==========================================
# 2. FIND ALL CSV FILES
# ==========================================
csv_files = glob.glob(f"{dataset_root}/*.csv")
print(f"Found {len(csv_files)} commodity files to process.")

# ==========================================
# 3. MAIN PROCESSING LOOP
# ==========================================
for file_path in csv_files:
    filename = os.path.basename(file_path)
    # Extract commodity name (Modify logic if needed based on your filenames)
    commodity_name = filename.split('_')[-1].replace('.csv', '').capitalize() 
    
    print(f"\n==========================================")
    print(f" LOADING MASTER FILE: {commodity_name}")
    print(f"==========================================")

    try:
        # Load the Master CSV
        master_df = pd.read_csv(file_path)
        master_df.columns = [c.lower().strip() for c in master_df.columns]

        if 'date' not in master_df.columns or 'price' not in master_df.columns:
            print(f"❌ Skipping {filename}: Column names mismatch. Found: {master_df.columns}")
            continue

        master_df['date'] = pd.to_datetime(master_df['date'], errors='coerce')
        master_df = master_df.dropna(subset=['date'])

        centers = master_df['centre_name'].unique()
        print(f"Found {len(centers)} centers: {centers}")

        # ==========================================
        # 4. CENTER-SPECIFIC TRAINING
        # ==========================================
        for center in centers:
            print(f"\n--- Training for {center} ({commodity_name}) ---")
            
            df_center = master_df[master_df['centre_name'] == center].copy()
            df_center = df_center.sort_values('date').set_index('date')
            df_center = df_center[['price']].rename(columns={'price': 'value'})
            
            df_center = df_center.asfreq('D', method='ffill')
            df_center = df_center.dropna()

            if len(df_center) < (prediction_days + 60):
                print(f"⚠️ Skipping {center}: Insufficient data ({len(df_center)} points)")
                continue

            # Split Data
            split_idx = int(len(df_center) * 0.8)
            train_data = df_center.iloc[:split_idx]
            test_data = df_center.iloc[split_idx:]

            # Scaling
            scaler = MinMaxScaler(feature_range=(0, 1))
            scaled_train = scaler.fit_transform(train_data.values)

            # Create Training Sequences
            x_train, y_train = [], []
            for x in range(prediction_days, len(scaled_train)):
                x_train.append(scaled_train[x-prediction_days:x, 0])
                y_train.append(scaled_train[x, 0])

            x_train, y_train = np.array(x_train), np.array(y_train)
            x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

            # Build LSTM Model
            model = Sequential()
            model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
            model.add(LSTM(units=50, return_sequences=False))
            model.add(Dense(units=25))
            model.add(Dense(1))
            model.compile(optimizer='adam', loss='mean_squared_error')

            # Train
            model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            # Save Artifacts
            save_dir = os.path.join(models_root, commodity_name, center)
            os.makedirs(save_dir, exist_ok=True)
            joblib.dump(scaler, os.path.join(save_dir, 'scaler.pkl'))
            model.save(os.path.join(save_dir, 'model.keras'))

            # --- Evaluation & Prediction ---
            total_data = pd.concat((train_data['value'], test_data['value']), axis=0)
            model_inputs = total_data[len(total_data) - len(test_data) - prediction_days:].values
            model_inputs = model_inputs.reshape(-1, 1)
            model_inputs = scaler.transform(model_inputs)

            x_test = []
            if len(model_inputs) > prediction_days:
                for x in range(prediction_days, len(model_inputs)):
                    x_test.append(model_inputs[x-prediction_days:x, 0])

                x_test = np.array(x_test)
                x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

                predicted_prices = model.predict(x_test, verbose=0)
                predicted_prices = scaler.inverse_transform(predicted_prices)

                # =========================================================
                # ✅ ADDED: RMSE and R² Calculation Block
                # =========================================================
                try:
                    # Align actual prices with predicted prices
                    actual_prices = test_data['value'].values
                    
                    # Ensure lengths match exactly before calculation
                    min_len = min(len(actual_prices), len(predicted_prices))
                    actual_prices = actual_prices[:min_len]
                    predicted_prices_calc = predicted_prices[:min_len]

                    rmse = math.sqrt(mean_squared_error(actual_prices, predicted_prices_calc))
                    r2 = r2_score(actual_prices, predicted_prices_calc)

                    print(f"📊 {center} Performance:")
                    print(f"   RMSE: {rmse:.2f}")
                    print(f"   R² Score: {r2:.4f}")
                    
                    # Optional: Warning for poor performance
                    if r2 < 0:
                        print("   ⚠️ Model is performing poorly (Negative R²). Consider more data or tuning.")

                except Exception as eval_error:
                    print(f"   ⚠️ Could not calculate metrics: {eval_error}")
                # =========================================================

                # --- Future Forecast ---
                future_predictions = []
                curr_batch = model_inputs[-prediction_days:].reshape(1, prediction_days, 1)

                for i in range(future_days):
                    pred_one = model.predict(curr_batch, verbose=0)[0]
                    future_predictions.append(pred_one)
                    curr_batch = np.append(curr_batch[:, 1:, :], [[pred_one]], axis=1)

                future_predictions = scaler.inverse_transform(future_predictions)
                
                last_date = df_center.index[-1]
                future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_days)

                # --- Plotting ---
                graph_save_dir = os.path.join(graphs_root, commodity_name)
                os.makedirs(graph_save_dir, exist_ok=True)
                
                plt.figure(figsize=(12, 6))
                plt.plot(test_data.index, test_data['value'], color='black', label="Actual Price")
                plt.plot(test_data.index[:len(predicted_prices)], predicted_prices, color='green', label="Test Predictions")
                plt.plot(future_dates, future_predictions, color='red', linestyle='--', label="6-Month Forecast")
                
                # Add text box with metrics to the plot
                stats_text = f'RMSE: {rmse:.2f}\n$R^2$: {r2:.2f}'
                plt.text(0.02, 0.95, stats_text, transform=plt.gca().transAxes, 
                         fontsize=10, verticalalignment='top', 
                         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

                plt.title(f"{commodity_name} Price Forecast - {center}")
                plt.xlabel("Date")
                plt.ylabel("Price")
                plt.legend()
                plt.grid(True)
                
                plt.savefig(os.path.join(graph_save_dir, f"{center}_forecast.png"))
                plt.close()
                print(f"✅ Saved model and graph for {center}")

    except Exception as e:
        print(f"❌ Critical Error processing file {file_path}: {e}")

print("\nAll centers processed.")

Found 17 commodity files to process.

 LOADING MASTER FILE: Toor
Found 109 centers: ['Alipurduar' 'Angul' 'Araria' 'Arwal' 'Asansol' 'Aurangabad' 'Balangir'
 'Balasore' 'Balurghat' 'Banka' 'Bankura' 'Barasat' 'Bardhaman' 'Baripada'
 'Baruipur' 'Begusarai' 'Berhampore' 'Berhampur' 'Bhadrak' 'Bhagalpur'
 'Bhojpur Ara' 'Bhubaneshwar' 'Bokaro' 'Boudhgarh' 'Buxar' 'Chinsurah'
 'Cooch Behar' 'Cuttack' 'Daltonganj' 'Darbhanga' 'Daudnagar Aurangabad'
 'Deoghar' 'Dhanbad' 'Dhenkanal' 'Gaya' 'Giridhi' 'Godda' 'Gopalganj'
 'Gumla' 'Howrah' 'Jagatsinghpur' 'Jajpur' 'Jalapiguri'
 'Jamshedpur (East Singbhum)' 'Jamtara' 'Jamui' 'Jehanabad' 'Jeypore'
 'Jhargram' 'Jharsuguda' 'Kaimur' 'Kalahandi' 'Kalimpong' 'Kandhamal'
 'Katihar' 'Kendrapara' 'Keonjhar' 'Khagaria' 'Kharagpur' 'Kishanganj'
 'Koderma' 'Kolkata' 'Krishnagar' 'Lakhisarai' 'Latehar' 'Lohardaga'
 'Madhubani' 'Malda' 'Malkangiri' 'Medhepura' 'Motihari' 'Munger'
 'Muzzafarpur' 'Nabarangpur' 'Nalanda' 'Nawada' 'Nayagarh' 'Nuapada'
 'Pakur' 'Pa

/Users/aryangahlot/.pyenv/versions/tfenv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


📊 Alipurduar Performance:
   RMSE: 2.07
   R² Score: 0.0000
✅ Saved model and graph for Alipurduar

--- Training for Angul (Toor) ---


/Users/aryangahlot/.pyenv/versions/tfenv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


📊 Angul Performance:
   RMSE: 2.45
   R² Score: -0.7672
   ⚠️ Model is performing poorly (Negative R²). Consider more data or tuning.
✅ Saved model and graph for Angul

--- Training for Araria (Toor) ---


...

📊 Vaishali Performance:
   RMSE: 1.89
   R² Score: 0.7446
✅ Saved model and graph for Vaishali

--- Training for West Champaran (Onion) ---


/Users/aryangahlot/.pyenv/versions/tfenv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


📊 West Champaran Performance:
   RMSE: 0.10
   R² Score: 0.0000
✅ Saved model and graph for West Champaran

--- Training for West Singhbhum (Onion) ---


/Users/aryangahlot/.pyenv/versions/tfenv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


📊 West Singhbhum Performance:
   RMSE: 0.86
   R² Score: 0.0050
✅ Saved model and graph for West Singhbhum

All centers processed.
